In [1]:
import json 


In [2]:
with open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gemini/rotowire/gemini_input.json.combined_results.json") as f:
    data = json.load(f)

In [3]:
pred_tables = {}
for i in data:
    pred_tables[i['key']] = i['result']

In [42]:
import re


def extract__response(text):

    tables = {}
    # Pattern explanation:
    #   - ^##\s*(.*?)\s*$ : a header line starting with '##' and capturing the title.
    #   - ((?:\|.*\n?)+)  : one or more lines that start with '|' (the table block).
    pattern = re.compile(
        r'^##\s*(.*?)\s*$\n((?:\|.*\n?)+)', 
        re.MULTILINE
    )
    
    matches = pattern.findall(text)
    for title, table in matches:
        tables[title.strip()] = table.strip()
    if 'Team Table' not in tables:
        tables['Team Table'] = ''
    if 'Player Table' not in tables:
        tables['Player Table'] = ''
    return tables['Team Table'], tables['Player Table']

ss = """##Team Table

lakjsdf

##Player Table
| Team             | Losses | Total points | Wins |
|-----------------|--------|--------------|------|
| Trail Blazers    | 25     | 109           | 48   |
| Suns             | 37     | 86            | 38   |

"""
print(extract__response(ss))

('', '| Team             | Losses | Total points | Wins |\n|-----------------|--------|--------------|------|\n| Trail Blazers    | 25     | 109           | 48   |\n| Suns             | 37     | 86            | 38   |')


In [43]:
def create_markdown_tables(data):
    # Replace the custom <NEWLINE> token with actual newline characters.
    data = data.replace("<NEWLINE>", "\n")
    
    # Split the data into sections using "Team:" and "Player:" markers.
    sections = data.split("Player:")
    
    # Extract team section (everything after "Team:" up to "Player:") if available.
    team_section_raw = sections[0].split("Team:")
    team_section = team_section_raw[1].strip() if len(team_section_raw) > 1 else ""
    
    # Extract player section (everything after "Player:") if available.
    player_section = sections[1].strip() if len(sections) > 1 else ""
    
    # Helper function to split a row by "|" while preserving empty cells.
    def process_row(row):
        # Split the row by the delimiter and strip whitespace from each cell.
        cells = [cell.strip() for cell in row.split("|")]
        # Remove leading and trailing empty cells due to delimiters.
        if cells and cells[0] == "":
            cells = cells[1:]
        if cells and cells[-1] == "":
            cells = cells[:-1]
        return cells
    
    # Helper function to create a markdown table from header and row cells.
    def markdown_table(header_cells, row_cells):
        header_row = "| " + " | ".join(header_cells) + " |"
        separator_row = "| " + " | ".join(["---"] * len(header_cells)) + " |"
        data_rows = ["| " + " | ".join(cells) + " |" for cells in row_cells]
        return "\n".join([header_row, separator_row] + data_rows)
    
    # Process team section.
    if team_section:
        team_lines = [line for line in team_section.split("\n") if line.strip()]
        if team_lines:
            team_header = process_row(team_lines[0])
            team_rows = [process_row(row) for row in team_lines[1:]]
            # Set the first header cell to "Team"
            if team_header:
                team_header[0] = "Team"
            team_table = markdown_table(team_header, team_rows)
        else:
            team_table = ""
    else:
        team_table = ""
    
    # Process player section.
    if player_section:
        player_lines = [line for line in player_section.split("\n") if line.strip()]
        if player_lines:
            player_header = process_row(player_lines[0])
            player_rows = [process_row(row) for row in player_lines[1:]]
            # Set the first header cell to "Player"
            if player_header:
                player_header[0] = "Player"
            player_table = markdown_table(player_header, player_rows)
        else:
            player_table = ""
    else:
        player_table = ""
    
    return team_table, player_table

# Example usage:
data = """Team: <NEWLINE> Player: <NEWLINE> |  | Assists | Defensive rebounds | 3-pointers attempted | Field goals attempted | Field goals made | Free throws made | Minutes played | Personal fouls | Points | Total rebounds | <NEWLINE> | Carmelo Anthony |  | 6 |  | 36 | 18 | 6 | 46 | 6 | 45 | 6 | <NEWLINE> | Kristaps Porzingis |  |  |  |  |  |  |  |  | 15 |  | <NEWLINE> | Courtney Lee |  |  |  |  |  |  |  |  | 17 | 9 | <NEWLINE> | Brandon Jennings | 11 |  |  |  |  |  |  |  | 18 |  | <NEWLINE> | Mindaugas Kuzminskas |  |  |  | 13 |  |  |  |  | 13 |  | <NEWLINE> | Justin Holiday |  |  | 13 |  |  |  |  |  | 15 |  | <NEWLINE> | Tim Hardaway Jr. |  |  |  |  |  |  |  |  | 19 |  | <NEWLINE> | Paul Millsap |  |  |  |  |  |  |  |  | 37 | 19 | <NEWLINE> | Dwight Howard |  |  |  |  |  |  |  |  | 19 |  | <NEWLINE> | Kent Bazemore |  |  |  |  |  |  |  |  | 24 |  | <NEWLINE> | Dennis Schroder | 15 |  |  |  |  |  |  |  | 23 |  | <NEWLINE> | Mike Dunleavy |  |  |  |  |  |  |  |  | 12 |  |"""
team_table, player_table = create_markdown_tables(data)
print("Team Table:")
print(team_table)
print("\nPlayer Table:")
print(player_table)


Team Table:


Player Table:
| Player | Assists | Defensive rebounds | 3-pointers attempted | Field goals attempted | Field goals made | Free throws made | Minutes played | Personal fouls | Points | Total rebounds |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Carmelo Anthony |  | 6 |  | 36 | 18 | 6 | 46 | 6 | 45 | 6 |
| Kristaps Porzingis |  |  |  |  |  |  |  |  | 15 |  |
| Courtney Lee |  |  |  |  |  |  |  |  | 17 | 9 |
| Brandon Jennings | 11 |  |  |  |  |  |  |  | 18 |  |
| Mindaugas Kuzminskas |  |  |  | 13 |  |  |  |  | 13 |  |
| Justin Holiday |  |  | 13 |  |  |  |  |  | 15 |  |
| Tim Hardaway Jr. |  |  |  |  |  |  |  |  | 19 |  |
| Paul Millsap |  |  |  |  |  |  |  |  | 37 | 19 |
| Dwight Howard |  |  |  |  |  |  |  |  | 19 |  |
| Kent Bazemore |  |  |  |  |  |  |  |  | 24 |  |
| Dennis Schroder | 15 |  |  |  |  |  |  |  | 23 |  |
| Mike Dunleavy |  |  |  |  |  |  |  |  | 12 |  |


In [44]:
with open("test.text", "r") as f:
    roto_text = f.read().split("\n")[:-1]

In [38]:
with open('test.data') as f:
    roto_tables = f.read().split("\n")[:-1]

In [39]:
roto_gold = {roto_text[i]: roto_tables[i] for i in range(len(roto_text))}

In [40]:
roto_id_to_tables = {str(i): roto_tables[i] for i in range(len(roto_text))}

In [49]:
input_tables = {}
for k, v in pred_tables.items():
    gold_team, gold_player = create_markdown_tables(roto_id_to_tables[k])
    print(v)
    print(extract__response(v))
    pred_team, pred_player = extract__response(v)

    input_tables[k] = {
        'gold_team': gold_team,
        'gold_player': gold_player,
        'pred_team': pred_team,
        'pred_player': pred_player
    }

##Team Table

| Team       | Losses | Total points | Wins |
|------------|--------|--------------|------|
| Hawks      | 12     | 95           | 46   |
| Magic      | 41     | 88           | 19   |


##Player Table

| Player          | Assists | Blocks | Points | Total rebounds | Steals |
|-----------------|---------|--------|--------|----------------|--------|
| Al Horford      | 4       |        | 17     | 13              | 2      |
| Jeff Teague     | 7       |        | 17     |                | 2      |
| Kyle Korver     |         |        |        |                |        |
| Nikola Vucevic |         |        | 21     | 15              |        |


('| Team       | Losses | Total points | Wins |\n|------------|--------|--------------|------|\n| Hawks      | 12     | 95           | 46   |\n| Magic      | 41     | 88           | 19   |', '| Player          | Assists | Blocks | Points | Total rebounds | Steals |\n|-----------------|---------|--------|--------|----------------|------

In [50]:
idx_to_table_id = {}
tabscore = {}
ii=0
for text, tables in input_tables.items():
    if tables['gold_team'] and tables['pred_team']:
        
        tabscore[ii] = [tables['gold_team'], tables['pred_team']]
        idx_to_table_id[ii] = text+"$$TEAM"
        ii+=1
    if tables['gold_player'] and tables['pred_player']:
        tabscore[ii] = [tables['gold_player'], tables['pred_player']]
        idx_to_table_id[ii] = text+"$$PLAYER"
        ii+=1

    

In [1]:
idx_to_table_id

NameError: name 'idx_to_table_id' is not defined

In [52]:
json.dump(tabscore, open("tabscore_input.json", "w"))
json.dump(idx_to_table_id, open("idx_to_table_id.json", "w"))

In [47]:
import pickle as pkl 
results = pkl.load(open("/home/turning/Jainit/TANQ/EVALUATION_OF_MODELS/gpt/rotowire/tabscore_results.pkl", "rb"))

In [8]:
with open('test.text') as f:
    roto_text = f.read().split("\n")[:-1]

In [11]:
with open("gemini_input.json", "w") as f:
    data = {i:t for i, t in enumerate(roto_text)}
    json.dump(data, f)